In [2]:
import pandas as pd
import time

k1 = 0.0013
mol_MDI = 0.0105
vol1 = 0.06
dt1 = 1
k1_calc = k1/vol1*mol_MDI*dt1

k2 = 0.00059
vol2 = 0.08
dt2 = 1
k2_calc = k2/vol2*mol_MDI*dt2

n1 = 10
n2 = 50

In [3]:
t0 = time.time()

PAA1 = [0 for i in range(n1)]
PBB1 = [0 for i in range(n1)]
PBA1 = [0 for i in range(n1)]
PAA1[0] = 1
PBB1[1] = 0.29
note = [[0]+PAA1[0:n1]+PBB1[0:n1]+PBA1[0:n1]]

t = 0
itr = 180*60

while (t <= itr):
    oldAA = PAA1*1
    oldBB = PBB1*1
    oldBA = PBA1*1

    # AA+BB -> AB
    for x1 in range(n1):
        for x2 in range(n1):
            d = k1_calc*oldAA[x1]*oldBB[x2]*4
            PAA1[x1] -= d
            PBB1[x2] -= d
            PBA1[min(x1+x2,n1-1)] += d
    # AB+AA -> AA
    for x1 in range(n1):
        for x2 in range(n1):
            d = k1_calc*oldAA[x1]*oldBA[x2]*2
            PAA1[x1] -= d
            PBA1[x2] -= d
            PAA1[min(x1+x2,n1-1)] += d
    # AB+BB -> BB
    for x1 in range(n1):
        for x2 in range(n1):
            d = k1_calc*oldBA[x1]*oldBB[x2]*2
            PBA1[x1] -= d
            PBB1[x2] -= d
            PBB1[min(x1+x2,n1-1)] += d
    # AB+AB -> AB
    for x1 in range(n1):
        for x2 in range(n1):
            d = k1_calc*oldBA[x1]*oldBA[x2]
            PBA1[x1] -= d
            PBA1[x2] -= d
            PBA1[min(x1+x2,n1-1)] += d

    # print(AA)
    # print(AB)
    # print(BB)

    t += dt1
    
    if t%60 == 0:
        note.append([t/60]+PAA1[0:n1]+PBB1[0:n1]+PBA1[0:n1])
        if t%600 == 0:
            print(t/60)
            print(PAA1[0:n1])
            print(PBB1[0:n1])
            print(PBA1[0:n1])

memo = pd.DataFrame(note)
memo.to_csv("step1_noFLAP.csv")

print(time.time()-t0)

10.0
[0.8701951790383359, 0.013519135238139305, 0.0002092683255579615, 3.234415576266959e-06, 4.994140760502074e-08, 7.705648269934964e-10, 1.1882447250656551e-11, 1.8314438018334969e-13, 2.8216459843382127e-15, 4.413607431981982e-17]
[0.0, 0.17122320295887036, 0.002661801144520597, 4.1216471739040686e-05, 6.371723542524212e-07, 9.839935625446246e-09, 1.5184377146681473e-10, 2.3417506537980557e-12, 3.60967926655997e-14, 5.648770253257005e-16]
[0.0, 0.09632220898895749, 0.0014932062744311503, 2.309693345193205e-05, 3.5682412196587104e-07, 5.5078452762806546e-09, 8.49617178632847e-11, 1.309883296277654e-12, 2.0185818809669004e-14, 3.1581370900176463e-16]
20.0
[0.7813558493338681, 0.03627933272877744, 0.0016815991357588891, 7.789153376912428e-05, 3.6063730840211e-06, 1.6692192222009137e-07, 7.724087098830454e-09, 3.5734751629271223e-10, 1.652941248044022e-11, 8.015281566044426e-13]
[0.0, 0.10431615287304746, 0.004846661934741163, 0.00022472242344854325, 1.0411369848557061e-05, 4.821237741

In [4]:
t0 = time.time()

PA2 = [0 for i in range(n2)]
PB2 = [0 for i in range(n2)]
PAA2 = [0 for i in range(n2)]
PBB2 = [0 for i in range(n2)]
PBA2 = [0 for i in range(n2)]
HS2 = [0 for i in range(n2)]

term_NCO = sum(PAA1[1:])*2+sum(PBA1)
term_OH = sum(PBB1)*2+sum(PBA1)

PAA2[0] = PAA1[0]   #MDI
PBB2[1] = 0.67   #BD
PA2[0] = term_NCO
PB2[0] = term_OH

print(term_NCO,term_OH)


note = [[0]+PA2+PB2+PAA2+PBB2+PBA2+HS2]

t = 0
itr = 48*60*60

while (t <= itr):
    oldAA = PAA2*1
    oldBB = PBB2*1
    oldBA = PBA2*1
    oldA = PA2*1
    oldB = PB2*1

    # AA+BB -> AB
    for y1 in range(n2):
        for y2 in range(n2):
            d = k2_calc*oldAA[y1]*oldBB[y2]*4
            PAA2[y1] -= d
            PBB2[y2] -= d
            PBA2[min(y1+y2,n2-1)] += d
    # AB+AA -> AA
    for y1 in range(n2):
        for y2 in range(n2):
            d = k2_calc*oldAA[y1]*oldBA[y2]*2
            PAA2[y1] -= d
            PBA2[y2] -= d
            PAA2[min(y1+y2,n2-1)] += d
    # AB+BB -> BB
    for y1 in range(n2):
        for y2 in range(n2):
            d = k2_calc*oldBA[y1]*oldBB[y2]*2
            PBA2[y1] -= d
            PBB2[y2] -= d
            PBB2[min(y1+y2,n2-1)] += d
    # AB+AB -> AB
    for y1 in range(n2):
        for y2 in range(n2):
            d = k2_calc*oldBA[y1]*oldBA[y2]
            PBA2[y1] -= d
            PBA2[y2] -= d
            PBA2[min(y1+y2,n2-1)] += d
    # A+BB -> B
    for y1 in range(n2):
        for y2 in range(n2):
            d = k2_calc*oldA[y1]*oldBB[y2]*2
            PA2[y1] -= d
            PBB2[y2] -= d
            PB2[min(y1+y2,n2-1)] += d
    # A+BA -> A
    for y1 in range(n2):
        for y2 in range(n2):
            d = k2_calc*oldA[y1]*oldBA[y2]
            PA2[y1] -= d
            PBA2[y2] -= d
            PA2[min(y1+y2,n2-1)] += d
    # B+BA -> A
    for y1 in range(n2):
        for y2 in range(n2):
            d = k2_calc*oldB[y1]*oldBA[y2]
            PB2[y1] -= d
            PBA2[y2] -= d
            PA2[min(y1+y2,n2-1)] += d
    # B+AA -> B
    for y1 in range(n2):
        for y2 in range(n2):
            d = k2_calc*oldB[y1]*oldAA[y2]*2
            PB2[y1] -= d
            PAA2[y2] -= d
            PB2[min(y1+y2,n2-1)] += d
    # A+B -> HS
    for y1 in range(n2):
        for y2 in range(n2):
            d = k2_calc*oldA[y1]*oldB[y2]
            PA2[y1] -= d
            PB2[y2] -= d
            HS2[min(y1+y2,n2-1)] += d
    

    t += dt2

    if t%60 == 0:
        note.append([t/60]+PA2+PB2+PAA2+PBB2+PBA2+HS2)
        if t%600 == 0:
            print(t/60)
            print(PA2[0:10])
            print(PB2[0:10])
            print(PAA2[0:10])
            print(PBB2[0:10])
            print(PBA2[0:10])
            print(HS2[0:10])

memo = pd.DataFrame(note)
memo.to_csv("step2_noFLAP.csv")

print(time.time()-t0)

0.40641881951669173 0.012673185694635002
10.0
[0.38241790574010165, 0.0005161858550658709, 2.077213271794831e-05, 5.912416063883891e-08, 2.789314430657727e-10, 1.0206297440380423e-12, 3.582553650297111e-15, 1.2025731427994028e-17, 3.9054941261733144e-20, 1.235920755323855e-22]
[0.012426309111958396, 0.02279219761428228, 3.0360564874645547e-05, 3.4822099606695593e-07, 1.187579864065821e-09, 4.490481442333664e-12, 1.5655312904518356e-14, 5.238437166440144e-17, 1.6971119396528556e-19, 5.360525807094548e-22]
[0.4543088077010337, 0.001188477554161899, 3.0971404337560243e-06, 8.057618006053328e-09, 2.0940244726374757e-11, 5.437598490316975e-14, 1.4110878433312727e-16, 3.6599023358562453e-19, 9.48826570164707e-22, 2.4588440950789253e-24]
[0.0, 0.5888552218499782, 0.0015404714598914971, 4.014451487117723e-06, 1.0444165631452162e-08, 2.7142518852093996e-11, 7.048173507323433e-14, 1.8290449557026437e-16, 4.743955208331739e-19, 1.229868224084067e-21]
[0.0, 0.052953390066799826, 0.0001381832257588